In [56]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [57]:
print('Program started')
sim.simxFinish(-1)  # just in case, close all opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected to remote API server')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive. You can guarantee this with (for example):
    sim.simxGetPingTime(clientID)

    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)
else:
    print('Failed connecting to remote API server')
print('Program ended')

Program started
Connected to remote API server
Program ended


In [58]:
class robot():
    
    def __init__(self, motor_names, frame_name=None):        
        self.client_id = self.open_connection()
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  None
        if frame_name is not None:
            self.frame = self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object):
        relative_handle = self._get_handler(relative_object)
        sim.simxSetObjectPosition(clientID, self.frame, relative_handle, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)

In [59]:
def move(name, init_velocity):
    r = robot([], frame_name='Quadricopter_target')
    # Time duration
    tf = 10
    # Time interval
    time_steps = linspace(0, tf, 500)

    robot_position = r.get_position()
    desired_position = r.get_object_position(name)

#     print(robot_position)
#     print(desired_position)


    a0 = robot_position
#     a1 = [0, 0, 1]
    a1 = init_velocity
    a2 = (np.dot(3,desired_position)-3*robot_position-np.dot(a1,2*tf))/tf**2
    a3 = (2*robot_position+np.dot(a1,tf)-np.dot(2,desired_position))/tf**3

    for t in time_steps:
            point_traj = a0 + np.dot(a1,t) + np.dot(a2,t*t) + np.dot(a3,t*t*t)
#             print(point_traj)
            r.set_position(point_traj, 'Cuboid8')

            time.sleep(tf/500)

    r.close_connection()

In [72]:
move('Sphere10',(0, 0, 1))

Robot connected
Connection closed


In [73]:
move('Sphere8',(-0.5, 0, 1))

Robot connected
Connection closed


In [74]:
move('Sphere9',(0, 0, 1))

Robot connected
Connection closed


In [75]:
move('Sphere5',(0, 0, 1))

Robot connected
Connection closed


In [76]:
move('Sphere6',(0, 2, 0))

Robot connected
Connection closed


In [77]:
move('Sphere7',(0, 2, 0))

Robot connected
Connection closed
